### Environment Preparation 

* prepare session and bucket 
* prepare necessary parameters 

In [ ]:
import sagemaker 
import boto3 
sess = sagemaker.Session()
BUCKET = sess.default_bucket()

In [ ]:
BUCKET

In [ ]:
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]


In [ ]:
!ls 


### Setup lambda functions and SQS 


We use a lambda function ```process_a2i_results``` to 
1. listening to SQS and process a2i results 
2. trigger retraining pipelines 

The following steps creates lambda functions 

* copying codes to s3 
* create lambda functions by codes on s3 
* setup environment variables 


In [ ]:
%%bash -s "$BUCKET" 
cd ./process_a2i_results
zip -r process_a2i_results.zip  .
aws s3 cp process_a2i_results.zip s3://$1/lambda/

In [ ]:
%store -r lambda_role_arn 

In [ ]:
lambda_role_arn

In [ ]:
import os
cwd = os.getcwd()
!aws lambda create-function --function-name process_a2i_results --zip-file fileb://$cwd/process_a2i_results/process_a2i_results.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role $lambda_role_arn

#### Configure lambda function - process_a2i_results
![configure environment variable](./content_image/setup_env_vars_for_lambda3.png)

In [ ]:
%store -r model_package_group_name 
%store -r pipeline_name

In [ ]:
bucket_key = "a2i-demo"
variables = f"MODEL_GROUP={model_package_group_name},BUCKET={BUCKET},PIPELINE={pipeline_name}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name process_a2i_results --environment "$env"

### Set up SQS 

* we use SQS to pass the information between lambda functions in this solution 
* set up SQS and attach it as a trigger to a lambda function 



In [ ]:
!aws lambda create-event-source-mapping --function-name process_a2i_results  --batch-size 10 \
--maximum-batching-window-in-seconds 60 --event-source-arn arn:aws:sqs:us-west-2:$account_id:a2itasks 
